In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("CUDA devices:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.10.0
CUDA devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Vision Transformers (Test)

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
import os

In [3]:
train_dir = r"SLangDataset\new_Blmark_data_combined"       # <-- Change this to your actual train directory
validation_dir = r"SLangDataset\BinaryLmarkBasedData\LMARK_merged_data_est" # <-- Change this to your actual validation directory

In [4]:
# Parameters
IMAGE_SIZE = 224  # ViT models typically use 224x224 images
BATCH_SIZE = 32
NUM_CLASSES = 25
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
# Load training and validation datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",  # One-hot encoded labels
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=False
)

Found 100000 files belonging to 25 classes.
Found 30000 files belonging to 25 classes.


In [6]:
# Optimize dataset performance with prefetching
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [7]:
preprocess_input = layers.Rescaling(1./255)

In [8]:
data_augmentation = keras.Sequential([
    layers.RandomRotation(0.1),
])

### Model

In [9]:
# URL for a pretrained ViT model on TF Hub
vit_model_url = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"  

# Create the TF Hub KerasLayer. Set trainable=True to fine-tune the ViT.
vit_layer = hub.KerasLayer(vit_model_url, trainable=True)

#### Building Model

In [10]:
# Build the model
inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = data_augmentation(inputs)      # Apply on-the-fly augmentation
x = preprocess_input(x)            # Normalize the images

# Extract features using the pretrained ViT
features = vit_layer(x)

# Add a dropout layer for regularization
x = layers.Dropout(0.5)(features)

# Final classification layer
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

# Create the model
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 768)               85798656  
                                                                 
 dropout (Dropout)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 25)                19225     
                                                                 
Total params: 85,817,881
Trainable params: 85,817,881
Non-tra

#### Compiling Model

In [11]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

#### Training Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',     # Monitor the validation loss
    patience=5,             # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

EPOCHS = 75  # Maximum number of epochs
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stopping]  # Add the early stopping callback here
)

Epoch 1/75


  11/3125 [..............................] - ETA: 98:06:58 - loss: 3.9559 - accuracy: 0.0767

#### Model Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize lists to store true labels and predictions.
all_labels = []
all_preds = []

# Iterate over the validation dataset and accumulate predictions and true labels.
for images, labels in val_ds:
    preds = model.predict(images)
    # Convert one-hot labels to class indices
    all_labels.extend(np.argmax(labels, axis=1))
    all_preds.extend(np.argmax(preds, axis=1))
    
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Print the classification report.
print("Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))

# Compute the confusion matrix.
cm = confusion_matrix(all_labels, all_preds)

# Plot the confusion matrix as a heatmap.
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=list(train_ds.class_names), 
            yticklabels=list(train_ds.class_names))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

#### Save Model

In [ ]:
model.save("asl_vit_classifier.h5")